In [46]:
import os
import pandas as pd

In [47]:
exp_root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/diagnosis"

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

In [48]:
df_index = pd.read_csv(os.path.join(exp_root, "exp_status.csv"))
df_index

,dataset,model,strategy,pred_file,rel_path,n,auroc,ci_low,ci_high,auc_std,status
0,Camelyon17,BLIP2-2.7b,eval_seed0/original_pretrained,/project/6101776/nanboy/MedVLMBench/log/diagno...,Camelyon17/BLIP2-2.7b/eval_seed0/original_pret...,920,0.517358,0.479656,0.555933,0.019424,OK
1,Camelyon17,BLIP2-2.7b,train_img-lora-lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,Camelyon17/BLIP2-2.7b/train_img-lora-lp_seed42,920,0.982491,0.975661,0.988340,0.003237,OK
2,Camelyon17,BLIP2-2.7b,train_lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,Camelyon17/BLIP2-2.7b/train_lp_seed42,920,0.981262,0.974411,0.987425,0.003409,OK
3,Camelyon17,BioMedCLIP,eval_seed0/original_pretrained,/project/6101776/nanboy/MedVLMBench/log/diagno...,Camelyon17/BioMedCLIP/eval_seed0/original_pret...,920,0.771692,0.741732,0.802931,0.015638,OK
4,Camelyon17,BioMedCLIP,train_img-lora-lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,Camelyon17/BioMedCLIP/train_img-lora-lp_seed42,920,0.986990,0.981275,0.992037,0.002773,OK
...,...,...,...,...,...,...,...,...,...,...,...
139,PAPILA,PubMedCLIP,train_img-lora-lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,PAPILA/PubMedCLIP/train_img-lora-lp_seed42,56,0.794326,0.658853,0.915159,0.066019,OK
140,PAPILA,PubMedCLIP,train_lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,PAPILA/PubMedCLIP/train_lp_seed42,56,0.517730,0.307061,0.734405,0.107715,OK
141,PAPILA,SigLIP,eval_seed0/original_pretrained,/project/6101776/nanboy/MedVLMBench/log/diagno...,PAPILA/SigLIP/eval_seed0/original_pretrained,56,0.517730,0.327592,0.734383,0.104283,OK
142,PAPILA,SigLIP,train_img-lora-lp_seed42,/project/6101776/nanboy/MedVLMBench/log/diagno...,PAPILA/SigLIP/train_img-lora-lp_seed42,56,0.713948,0.533528,0.876771,0.086208,OK


In [49]:
bootstrap_indices = {}

In [50]:
import tqdm
import json
import numpy as np
from sklearn.metrics import roc_auc_score

seed = 42
rng = np.random.default_rng(seed)
n_bootstraps = 1000

for i in tqdm.tqdm(range(len(df_index))):
    item = df_index.iloc[i]

    path = item["rel_path"]
    dataset = item["dataset"]
    each_example_file = os.path.join(exp_root, path, "predictions.json")
    assert os.path.exists(each_example_file)
    with open(each_example_file) as fp:
        predictions = json.load(fp)
    if isinstance(predictions, list):
        y_true = np.concatenate([np.array(row["true_label"], dtype=np.int8) for row in predictions])
        y_score = np.concatenate([np.array(row["pred_score"], dtype=np.float32) for row in predictions])
    else:
        y_true = np.array(predictions["true_label"], dtype=np.int8)
        y_score = np.array(predictions["pred_score"], dtype=np.float32)
    assert len(y_true) == len(y_score)
    # break
    save_file = os.path.join(exp_root, path, "bootstrap_result.csv")

    bootstrap_results = []

    # generate boot indices for each dataset
    if dataset not in bootstrap_indices.keys():
        indices = []
        for _ in range(n_bootstraps):
            indices.append(rng.integers(0, len(y_score), len(y_score)))
        bootstrap_indices[dataset] = indices

    for _ in range(n_bootstraps):
        sample_idx = bootstrap_indices[dataset][_]

        auc = roc_auc_score(y_true[sample_idx], y_score[sample_idx])

        bootstrap_results.append({"auc": auc})
    df_boot = pd.DataFrame(bootstrap_results)
    # break
    df_boot.to_csv(save_file, index=False)

100%|██████████| 144/144 [03:08<00:00,  1.31s/it]
